# Pytorch의 nn.Embedding
- Pytorch의 Embedding Layer는 word2vec과 마찬가지로 word embedding vector를 찾는 **Lookup Table**이다.
    - 단어의 **정수의 고유 index**가 입력으로 들어오면 Embedding Layer의 **그 index의 Vector**를 출력한다.
    - 모델이 학습되는 동안 모델이 풀려는 문제에 맞는 값으로 Embedding Layer의 vector들이 업데이트 된다.
    - Word2Vec의 embedding vector 학습을 nn.Embedding은 자신이 포함된 모델을 학습 하는 과정에서 한다고 생각하면 된다.

In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
from torchinfo import summary

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [2]:
e_layer = nn.Embedding(
    num_embeddings=10,   #vocab size (총 단어 갯수)
    embedding_dim=5,  #embedding vector의 차원
    padding_idx=0,  #padding 토큰의 index를 지정 (0인경우 pad는 자리만 채운 토큰이므로 학습하지않게함)  [PAD] 토큰 : 문장들의 토큰 갯수를 맞추기 위해서 사용하는 토큰, 모든 문장의 토큰수를 10개로 할 경우 10개가 안되는 토큰은 [PAD] 토큰으로 채움
)

print(e_layer)  # [10:단어수, 5:embedding차원]
e_layer.weight # word embedding vector들

Embedding(10, 5, padding_idx=0)


Parameter containing:
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 4.1567e-01, -4.1612e-01,  1.4438e+00, -4.3451e-01,  4.4722e-01],
        [ 1.2987e+00,  2.2073e+00,  1.2578e+00, -1.0437e+00,  7.5841e-01],
        [ 7.8317e-01, -1.6177e+00, -1.8964e+00,  9.7235e-02,  8.5216e-01],
        [ 1.1935e+00,  1.4814e-01, -6.6420e-01, -1.7455e-01,  1.4450e+00],
        [-6.3060e-01, -4.3839e-01, -1.0186e+00,  4.8676e-01,  6.0084e-01],
        [ 5.0924e-01,  1.7963e+00,  2.0019e-01,  7.1484e-01, -1.8524e-03],
        [-6.8214e-01,  1.6606e+00,  1.4089e-01,  9.6041e-01, -1.3932e+00],
        [ 2.9771e-01,  3.9424e-01, -6.8769e-01,  8.0933e-01,  2.5316e-01],
        [ 2.0457e-01,  6.9192e-01, -1.3371e+00,  4.8452e-01,  1.2761e+00]],
       requires_grad=True)

In [3]:
#입력 값 - 정수 tensor(LongTensor-int64)를 입력
## 한개 문서 : [1, 10, 7, 5] -> 토큰 index 문서 구성하는 토큰 idx들을 1차원으로 묶어서 전달

input_data = torch.tensor([[1, 3, 2, 7]], dtype=torch.int64)
e = e_layer(input_data)
print(e.shape) # [1, 4, 5] -> 문서수, 토큰수, embedding vector

torch.Size([1, 4, 5])


# 네이버 영화 댓글 감성분석(Sentiment Analysis)

## 감성분석(Sentiment Analysis) 이란
입력된 텍스트가 **긍적적인 글**인지 **부정적인**인지 또는 **중립적인** 글인지 분석하는 것을 감성(감정) 분석이라고 한다.   
이를 통해 기업이 고객이 자신들의 기업 또는 제품에 대해 어떤 의견을 가지고 있는지 분석한다.

# Dataset, DataLoader 생성

## Korpora에서 Naver 영화 댓글 dataset 가져오기
- https://ko-nlp.github.io/Korpora/ko-docs/corpuslist/nsmc.html
- http://github.com/e9t/nsmc/
    - input: 영화댓글
    - output: 0(부정적댓글), 1(긍정적댓글)
### API
- **corpus 가져오기**
    - `Korpora.load('nsmc')`
- **text/label 조회**
    - `corpus.get_all_texts()` : 전체 corpus의 text들을 tuple로 반환
    - `corpus.get_all_labels()`: 전체 corpus의 label들을 list로 반환
- **train/test set 나눠서 조회**
    - `corpus.train`
    - `corpus.test`
    - `LabeledSentenceKorpusData` 객체에 text와 label들을 담아서 제공.
        - `LabeledSentenceKorpusData.texts`: text들 tuple로 반환.
        - `LabeledSentenceKorpusData.labels`: label들 list로 반환.

## 데이터 로딩

In [4]:
import os
import time

from Korpora import Korpora

corpus = Korpora.load('nsmc')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /Users/seong-eunjin/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at /Users/

In [5]:
all_input = corpus.get_all_texts()  # 댓글들 전체
all_labels = corpus.get_all_labels() # output 0 부정 / 1 긍정

print(all_input[:5])
print(all_labels[:5])

('아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다')
[0, 1, 0, 0, 1]


In [19]:
corpus.train

NSMC.train: size=150000
  - NSMC.train.texts : list[str]
  - NSMC.train.labels : list[int]

In [20]:
corpus.test

NSMC.test: size=50000
  - NSMC.test.texts : list[str]
  - NSMC.test.labels : list[int]

## 토큰화
1. 형태소 단위 token화(분절)를 먼저 한다.
    - konlpy로 token화 한 뒤 다시 한 문장으로 만든다.
2. 1에서 처리한 corpus를 BPE 로 token화
   
### 전처리 함수

#### 형태소 단위 분절

In [7]:
from konlpy.tag import Okt, Mecab
import string
import re

def text_preprocessing(text):
    """
    1. 영문 -> 소문자로 변환
    2. 구두점 제거
    3. 형태소 기반 토큰화
    4. 형태소로 토큰화 한 뒤 다시 하나의 문자열로 묶어서 반환.
    """
    text = text.lower()
    text = re.sub(f'[{string.punctuation}]', ' ', text)
    tokens = Okt().morphs(text, stem=True) # stem 원형복원
    return ' '.join(tokens)

In [18]:
print(text_preprocessing('아 더빙.. 진짜 짜증나네요 목소리'))

train_inputs = corpus.train.texts
train_texts = [text_preprocessing(txt) for txt in train_inputs]
train_labels = corpus.train.labels

test_input = corpus.test.texts
test_texts = [text_preprocessing(txt) for txt in test_input]
test_labels = corpus.test.labels

# 전처리 된 input 합치기
all_texts = train_texts + test_texts

아 더빙 진짜 짜증나다 목소리


In [22]:
import pickle
import os
os.makedirs('datasets/nsmc', exist_ok=True)
with open('datasets/nsmc/train.pkl', 'wb') as fw :
    pickle.dump({'input':train_texts, 'output':train_labels}, fw)

with open('datasets/nsmc/testset.pkl', 'wb') as fw2 :
    pickle.dump({'input':test_texts, 'output':test_labels}, fw2)


### 토큰화
- Subword 방식 토큰화 적용
- Byte Pair Encoding 방식으로 huggingface tokenizer 사용
    - BPE: 토큰을 글자 단위로 나눈뒤 가장 자주 등장하는 글자 쌍(byte paire)를 찾아 합친뒤 어휘사전에 추가한다.
    - https://huggingface.co/docs/tokenizers/quicktour
    - `pip install tokenizers`

In [14]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

vocab_size = 30000
min_frequency = 5

tokenizer = Tokenizer(
    BPE(unk_token='[UNK]')
)
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(
    vocab_size=vocab_size,
    min_frequency=min_frequency,
    special_tokens=['[PAD]', '[UNK]'],
    continuing_subword_prefix='##'
)

#학습
tokenizer.train_from_iterator(all_texts, trainer=trainer) #학습코드가 메모리에 있을때 사용

In [ ]:
tokenizer.get_vocab_size()

26738

In [38]:
print(f'[PAD]의 id : {tokenizer.token_to_id('[PAD]')}')
print(tokenizer.id_to_token(1986))

[PAD]의 id : 0
아


In [37]:
# 문장 토큰화
all_texts[1023]
r = tokenizer.encode(all_texts[0])
r.ids

[1986, 5881, 5426, 5667, 6087]

In [39]:
tokenizer.decode([1986, 5881, 5426, 5667, 6087])

'아 더빙 진짜 짜증나다 목소리'

In [16]:
os.makedirs('saved_model/nsmc', exist_ok=True)
tokenizer.save('saved_model/nsmc/tokenizer.json')

In [24]:
load_tokenizer = Tokenizer.from_file('saved_model/nsmc/tokenizer.json')
load_tokenizer.encode(all_texts[0]).tokens

['아', '더빙', '진짜', '짜증나다', '목소리']

## Dataset, DataLoader 생성

In [61]:
import torch
from torch.utils.data import Dataset, DataLoader

class NSMCDataset(Dataset):
    def __init__(self, texts, labels, max_length, tokenizer):
        """
        texts: list - 댓글 목록. 리스트에 댓글들을 담아서 받는다. ["댓글", "댓글", ...]
        labels: list - 댓글 감정 목록. 
        max_length: 개별 댓글의 token 개수. 모든 댓글의 토큰수를 max_length에 맞춘다.
        tokenizer: Tokenizer
        """
        self.max_length = max_length
        self.tokenizer = tokenizer
        self.texts = [self.__pad_token_sequences(tokenizer.encode(text).ids) for text in texts]
        self.labels = labels

    ###########################################################################################
    # id로 구성된 개별 문장 tokenizer list를 받아서 패딩 추가 [20, 2, 1] => [20, 2, 1, 0, 0, 0, ..]
    ############################################################################################
    def __pad_token_sequences(self, token_sequences):
        """
        id로 구성된 개별 문서(댓글)의 token_id list를 받아서 max_length 길이에 맞추는 메소드
        max_length 보다 토큰수가 적으면 [PAD] 추가, 많으면 max_length 크기로 줄인다.
            ex) [20, 2, 1] => [20, 2, 1, 0, 0, 0, ..]
        """
        pad_token = self.tokenizer.token_to_id('[PAD]')
        seq_len = len(token_sequences) # 문장의 토큰 갯수
        result = None
        if seq_len > self.max_length :
            result = token_sequences[:self.max_length]
        else :
            result = token_sequences + ([pad_token] * (self.max_length - seq_len))
        return result
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        """
        idx 번째 text와 label을 학습 가능한 type으로 변환해서 반환
        Parameter
            idx: int 조회할 index
        Return
            tuple: (torch.LongTensor, torch.FloatTensor) - 댓글 토큰_id 리스트, 정답 Label
        """
        txt = self.texts[idx] # idx번째 댓글
        label = self.labels[idx] #idx번째 정답
        # (input, output) : input : embedding layer에 입력으로 들어감 -> LongTensor
        # output : [label] loss함수에 입력할 때 (batch, 1)
        return (torch.tensor(txt, dtype=torch.int64), torch.tensor([label], dtype=torch.float32))

# BCELoss() : 정답 shape (batch,1)
# CrossEntropyLoss() : 정답 shape (batch,)
    

In [32]:
a = [ len(tokenizer.encode(text)) for text in all_texts]  # 모든 문장들의 토큰 갯수 리스트
a[:5]

[5, 11, 10, 9, 22]

In [37]:
min(a), max(a)

(0, 89)

In [62]:
## dataset 생성

import numpy as np
np.quantile(a, q=0.9)

# max_token 수를 30개로 세팅
MAX_TOKEN = 30
train_set = NSMCDataset(train_texts, train_labels, MAX_TOKEN, tokenizer)
test_set = NSMCDataset(test_texts, test_labels, MAX_TOKEN, tokenizer)

train_set[10]

(tensor([  540, 11354,   506,  2408,  5414,  5426,  2408,  5414,   119,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 tensor([1.]))

In [64]:
# dataloader
BATCH_SIZE = 64
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE)

In [50]:
len(train_loader), len(test_loader)

(2343, 782)

# 모델링
- Embedding Layer를 이용해 Word Embedding Vector를 추출한다.
- LSTM을 이용해 Feature 추출
- Linear + Sigmoid로 댓글 긍정일 확률 출력
  
![outline](figures/rnn/RNN_outline.png)

## 모델 정의

In [26]:
import torch
import torch.nn as nn
from torchinfo import summary
import numpy as np

device = 'cuda' if torch.cuda.is_available() else "cpu"

In [ ]:
class NaverCommentClassifier(nn.Module) :

    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, bidirectional=True, droupout_rate=0.2) :
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings = vocab_size, # vocab 총 단어 수
            embedding_dim=embedding_dim, #embedding vector의 차원 수
            padding_idx = 0  #[PAD]의 index padding에 대한 weight는 학습하지않는다
        )

        # embedding layer의 출력 shape (64:batch, seq_len:문장의 토큰수, embedding vector 차원수) -> (64, 30, embedding_dim)
        self.lstm = nn.LSTM(
            input_size = embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=bidirectional,
            dropout=droupout_rate
        )

        self.dropout = nn.Dropout(droupout_rate)
        # 입력 : lstm의 마지막 timestep의 output
        if bidirectional == True :
            i_features = hidden_size * 2
        else :
            i_features = hidden_size
        self.classifier = nn.Linear(i_features, 1) # out_features : 1 - 긍정일 확률 
        self.sigmoid = nn.Sigmoid()

    def forward(self, X) :
        embedding_vector = self.embedding(X)
        # embedding_vector : [batch_size, seq_len, embedding_dim] -> [seq_len, batch_size, embedding]
        embedding_vector = embedding_vector.transpose(1, 0)  # batch축과 seq_len 축 바꾸기
        output, _ = self.lstm(embedding_vector)  # output = [seq_len, batch_size, hidden_size] -> 마지막 output 추출
        output = output[-1]
        output = self.dropout(output)
        output = self.classifier(output)
        last_output = self.sigmoid(output)
        return last_output

In [61]:
a = torch.randn((3, 2, 5))
print(a.shape)
b = a.transpose(1,0)  # 위치 바꾸기
b.shape
c = a.permute(2, 0, 1)

torch.Size([3, 2, 5])


## 모델 생성

In [28]:
model = NaverCommentClassifier(
     vocab_size=tokenizer.get_vocab_size(),
     embedding_dim=100,
     hidden_size=32,
     num_layers=2,
     bidirectional=True,
     droupout_rate=0.3
)

In [38]:
# 학습 전에 추론
X, y = next(iter(train_loader))
y_hat = model(X)
print(y_hat)

tensor([[0.5323],
        [0.5309],
        [0.5148],
        [0.5180],
        [0.5177],
        [0.5227],
        [0.5275],
        [0.5209],
        [0.5332],
        [0.5259],
        [0.5181],
        [0.5197],
        [0.5209],
        [0.5209],
        [0.5152],
        [0.5213],
        [0.5161],
        [0.5211],
        [0.5202],
        [0.5262],
        [0.5268],
        [0.5276],
        [0.5241],
        [0.5188],
        [0.5280],
        [0.5250],
        [0.5199],
        [0.5212],
        [0.5257],
        [0.5373],
        [0.5329],
        [0.5275],
        [0.5273],
        [0.5227],
        [0.5214],
        [0.5261],
        [0.5164],
        [0.5309],
        [0.5232],
        [0.5203],
        [0.5236],
        [0.5174],
        [0.5335],
        [0.5322],
        [0.5241],
        [0.5274],
        [0.5116],
        [0.5244],
        [0.5414],
        [0.5161],
        [0.5199],
        [0.5245],
        [0.5116],
        [0.5223],
        [0.5201],
        [0

## 학습

### Train/Test 함수 정의

In [74]:
def train(model, dataloader, loss_fn, optimizer, device='cpu'):
    # 1 epoch 학습
    model = model.to(device)
    model.train()
    loss_list = []
    for X, y in dataloader :
        X, y = X.to(device), y.to(device)
        # 추론
        pred = model(X)
        # loss 계산
        loss = loss_fn(pred, y)
        # gradient 계산
        loss.backward()
        # 파라미터 업데이트
        optimizer.step()
        # 파라미터 초기화
        optimizer.zero_grad()
        loss_list.append(loss.item())
    return sum(loss_list)/len(dataloader)  # 1 에폭 train loss: step loss들 평균을 반환.
    
def test(model, dataloader, loss_fn, device="cpu"):
    model = model.to(device)
    model.eval()
    loss_list = []
    acc_list = 0
    with torch.no_grad() :
        for X, y in dataloader :
            X, y = X.to(device), y.to(device)
            pred_proba = model(X) #  양성일 확률
            pred_label = (pred_proba > 0.5).type(torch.int32) # bool -> int (False: 0, True: 1)
            loss = loss_fn(pred_proba, y)
            loss_list.append(loss.item())
            acc_list += (y == pred_label).sum().item()
        return sum(loss_list), acc_list/len(dataloader.dataset) # 검증 loss, accuracy 



### Train

In [50]:
lr = 0.01
epochs = 1

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [75]:
for epoch in range(epochs) :
    #학습
    train_loss = train(model, train_loader, loss_fn, optimizer, device)
    val_loss, val_acc = test(model, test_loader, loss_fn, device)
    print(f'train loss = {train_loss}, val_loss = {val_loss}, val_acc = {val_acc}')

train loss = 0.28239506408031606, val_loss = 297.4188789278269, val_acc = 0.83802


## 모델저장

In [77]:
torch.save(model, 'saved_model/nsmc_model.pt')

# 서비스

## 전처리 함수들

In [80]:
from konlpy.tag import Okt
okt = Okt()

def text_preprocessing(text):
    # 한 문장(문서) 전처리
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]+", ' ', text)
    return ' '.join(okt.morphs(text, stem=True))

In [81]:
def pad_token_sequences(token_sequences, max_length):
    """padding 처리 메소드."""
    pad_token = tokenizer.token_to_id('[PAD]')  
    seq_length = len(token_sequences)           
    result = None
    if seq_length > max_length:                 
        result = token_sequences[:max_length]
    else:                                            
        result = token_sequences + ([pad_token] * (max_length - seq_length))
    return result

In [87]:
def predict_data_preprocessing(text_list):
    """
    모델에 입력할 수있는 input data를 생성
    Parameter:
        text_list: list - 추론할 댓글리스트
    Return
        torch.LongTensor - 댓글 token_id tensor
    """
    text_list = [text_preprocessing(text) for text in text_list]
    # 토큰화
    tokens = [ tokenizer.encode(text).ids for text in text_list]
    # 토큰 + 패딩
    pad_tokens = [pad_token_sequences(token, MAX_TOKEN) for token in tokens]
    return torch.tensor(pad_tokens, dtype=torch.int64)

## 추론

In [88]:
comment_list = ["아 진짜 재미없다.", "여기 식당 먹을만 해요", "이걸 영화라고 만들었냐?", "기대 안하고 봐서 그런지 괜찮은데.", "이걸 영화라고 만들었나?", "아! 뭐야 진짜.", "재미있는데.", "연기 짱 좋아. 한번 더 볼 의향도 있다.", "뭐 그럭저럭"]
input_tensor = predict_data_preprocessing(comment_list)
input_tensor.shape

torch.Size([9, 30])

In [95]:
# 추론
model.eval()
model.to(device)
with torch.no_grad() :
    pred = model(input_tensor)
    for txt, prob in zip(comment_list, pred) :
        label = '긍정적 댓글' if prob.item() > 0.5 else '부정적 댓글'
        p = prob.item() if prob.item() > 0.5 else (1-prob).item()
        print(txt, label, p)

아 진짜 재미없다. 부정적 댓글 0.9841747879981995
여기 식당 먹을만 해요 긍정적 댓글 0.7784380316734314
이걸 영화라고 만들었냐? 부정적 댓글 0.9288060069084167
기대 안하고 봐서 그런지 괜찮은데. 긍정적 댓글 0.8463358283042908
이걸 영화라고 만들었나? 부정적 댓글 0.9288060069084167
아! 뭐야 진짜. 부정적 댓글 0.8867478966712952
재미있는데. 긍정적 댓글 0.9645829200744629
연기 짱 좋아. 한번 더 볼 의향도 있다. 긍정적 댓글 0.9739835858345032
뭐 그럭저럭 부정적 댓글 0.9695420861244202


In [101]:
comment_list = [input("댓글 :")]
input_tensor = predict_data_preprocessing(comment_list)
# 추론
model.eval()
model.to(device)
with torch.no_grad() :
    pred = model(input_tensor)
    for txt, prob in zip(comment_list, pred) :
        label = '긍정적 댓글' if prob.item() > 0.5 else '부정적 댓글'
        p = prob.item() if prob.item() > 0.5 else (1-prob).item()
        print(txt, label, p)

킥 부정적 댓글 0.9816281795501709
